In [39]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn import datasets
from sklearn.svm import LinearSVC
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC

1. Data visualization
2. Data pre-processing
3. Model

In [2]:
train_data_path = ('./feedback-prize-english-language-learning/train.csv')

In [3]:
def load_data(path):
    corpus = pd.read_csv(path)
    return corpus

## Data visualization

In [4]:
df = load_data(train_data_path)

In [5]:
df['full_text'][2352]

"Dear friend\n\nmy name is STUDENT_NAME\n\nam coming from LOCATION_NAME.\n\ni speak LOCATION_NAME and LANGUAGE_NAME.\n\ni am a student to SCHOOL_NAME to 8grade.\n\nThe community is very important. Because he teacher people a good idea.\n\nex: my idea is to respect the teacher and other people.\n\nI don't no speak English good.\n\nBecause to my contry no English.\n\ni want to be a good student in English and other .\n\nThank's"

### Data pre processing

In [6]:
def tokenize_data(data):
    tokens = word_tokenize(data)
    return tokens

In [7]:
def lowercase_data(data):
    lowercase = np.char.lower(data)
    return lowercase

In [8]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-.,/:;<=>?@[\]^_`{|}~\n"
    no_punctuation = data
    for i in symbols:
        no_punctuation = np.char.replace(no_punctuation, i, ' ')
        no_punctuation = np.char.replace(no_punctuation, "'", "")        
        
    return no_punctuation

In [9]:
def numbers_to_words(data):
    newData = []
    for i in data:
        if i != str(i):
            num = num2words(i)
            newData.append(num)
        else:
            newData.append(i)
    return newData

In [10]:
def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    words = [w for w in data if not w in stop_words]
    return words

In [11]:
def lemmatize_words(data):
    a = []
    lemmatizer = WordNetLemmatizer()
    for i in data:
        lemmatized_word = lemmatizer.lemmatize(i)
        a.append(lemmatized_word)
    return a

In [12]:
def data_cleanning(data):
    df = data
    # Loop through data and clean it.
    for i in data['full_text']:
       
        ndata = tokenize_data(i)
        ndata = remove_punctuation(ndata)
        ndata = lowercase_data(ndata)
        ndata = numbers_to_words(ndata)
        ndata = remove_stop_words(ndata)
        ndata = lemmatize_words(ndata)
        ndata = numbers_to_words(ndata)
        ndata = ' '.join(ndata)
        ndata = remove_punctuation(ndata)
        
        df = df.replace(i, ndata)

    return pd.DataFrame(df)

In [13]:
def split_test_validation_byID(data):    
    train_d, val_d = train_test_split(data, test_size=0.2, random_state=1)
    train_d = pd.DataFrame(train_d)
    val_d = pd.DataFrame(val_d)
    return train_d, val_d

In [63]:
def tf_idf(data):
    
    vectorizer = TfidfVectorizer(max_features=2)
    tfidf = vectorizer.fit_transform(data)
    words = vectorizer.get_feature_names_out()
    
    return tfidf, words

In [59]:
def modelfor_Vocabulary_pipeline(data):
    dataf = load_data(data)
    

    # x_train, x_val, y_train, y_val = split_test_validation(train_dataPath)
    train_data, validation_data = split_test_validation_byID(dataf)

    
    df = data_cleanning(train_data[:10])
    x, words = tf_idf(df['full_text'])
   
    y = train_data['vocabulary']
    
#     svm_clf = LinearSVC()
#     svm_clf.fit(x,Y)
    return x

In [60]:
x = modelfor_Vocabulary_pipeline(train_data_path)

In [62]:
print(x)

  (0, 0)	0.5189579373112586
  (0, 1)	0.8547997773172638
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 1)	1.0
  (5, 0)	0.15005905719608004
  (5, 1)	0.9886770349074685
  (6, 1)	1.0
  (7, 0)	0.8068927113964852
  (7, 1)	0.5906980212386261
  (8, 0)	0.7281817737506422
  (8, 1)	0.6853840561155247
  (9, 0)	0.15005905719608004
  (9, 1)	0.9886770349074685
